<a href="https://colab.research.google.com/github/ysnvatsal/DataScience/blob/main/Auto_Tagging_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective

Build a model to automatically predict tags for a given a StackExchange question by using the text of the question.
![alt text](https://cdn.sstatic.net/Sites/stackoverflow/company/img/logos/se/se-logo.svg?v=d29f0785ebb7)

__Dataset Specs__: Over 85,000 questions

[Download Link](https://www.kaggle.com/stackoverflow/statsquestions#Questions.csv)

__License__

All Stack Exchange user contributions are licensed under [CC-BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/) with [attribution required](http://blog.stackoverflow.com/2009/06/attribution-required/).

<br>

***

In [26]:
# optional step (if you are working on colab)
# mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Steps to Follow



1. Load Data and Import Libraries
2. Text Cleaning
3. Merge Tags with Questions
4. Dataset Preparation
5. Text Representation
6. Model Building
    1. Define Model Architecture
    2. Train the Model
7. Model Predictions
8. Model Evaluation



# Load Data and Import Libraries

In [28]:
# for string matching
import re

# for reading data
import pandas as pd

# for handling html data
from bs4 import BeautifulSoup

# for visualization
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 200)

In [ ]:
# extract data from the ZIP file
!unzip 'statsquestions.zip'

In [29]:
# load the stackoverflow questions dataset
questions_df = pd.read_csv('Questions.csv',encoding='latin-1')

# load the tags dataset
tags_df = pd.read_csv('Tags.csv')

### Data Dictionary

1. Id: Question ID
2. OwnerUserId: User ID
3. CreationDate: Date of posting question
4. Score: Count of Upvotes received by the question
5. Title: Title of the question
6. Body: Text body of the question

In [30]:
#print first 5 rows
questions_df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,6,5.0,2010-07-19T19:14:44Z,272,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach..."
1,21,59.0,2010-07-19T19:24:36Z,4,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...
2,22,66.0,2010-07-19T19:25:39Z,208,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n
3,31,13.0,2010-07-19T19:28:44Z,138,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests...."
4,36,8.0,2010-07-19T19:31:47Z,58,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ..."


# Text Cleaning

Let's define a function to clean the text data.

In [31]:
def cleaner(text):

  # take off html tags
  text = BeautifulSoup(text).get_text()

  # fetch alphabetic characters
  text = re.sub("[^a-zA-Z]", " ", text)

  # convert text to lower case
  text = text.lower()

  # split text into tokens to remove whitespaces
  tokens = text.split()

  return " ".join(tokens)

In [32]:
# call preprocessing function
questions_df['cleaned_text'] = questions_df['Body'].apply(cleaner)

In [9]:
questions_df['Body'][1]

"<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\nareas are a lot larger than condensed\nurban areas. Is there a need to account for the area size difference?</li>\n<li>if let's say I have census data\ndating back to 4 - 5 census periods,\nhow far can i forecast it into the\nfuture?</li>\n<li>if some of the census zone change\nlightly in boundaries, how can i\naccount for that change?</li>\n<li>What are the methods to validate\ncensus forecasts? for example, if i\nhave data for existing 5 census\nperiods, should I model the first 3\nand test it on the latter two? or is\nthere another way?</li>\n<li>what's the state of practice in\nforecasting census data, and what are\nsome of the state of the art methods?</li>\n</ul>\n"

In [10]:
questions_df['cleaned_text'][1]

'what are some of the ways to forecast demographic census with some validation and calibration techniques some of the concerns census blocks vary in sizes as rural areas are a lot larger than condensed urban areas is there a need to account for the area size difference if let s say i have census data dating back to census periods how far can i forecast it into the future if some of the census zone change lightly in boundaries how can i account for that change what are the methods to validate census forecasts for example if i have data for existing census periods should i model the first and test it on the latter two or is there another way what s the state of practice in forecasting census data and what are some of the state of the art methods'

# Merge Tags with Questions

Let's now explore the tags data.

In [11]:
tags_df.head()

,Id,Tag
0,1,bayesian
1,1,prior
2,1,elicitation
3,2,distributions
4,2,normality


In [12]:
# count of unique tags
len(tags_df['Tag'].unique())

1315

In [13]:
tags_df['Tag'].value_counts()

r                       13236
regression              10959
machine-learning         6089
time-series              5559
probability              4217
                        ...  
fmincon                     1
doc2vec                     1
sympy                       1
adversarial-boosting        1
corpus-linguistics          1
Name: Tag, Length: 1315, dtype: int64

In [33]:
# remove "-" from the tags
tags_df['Tag']= tags_df['Tag'].apply(lambda x:re.sub("-"," ",x))

In [34]:
# group tags Id wise
tags_df = tags_df.groupby('Id').apply(lambda x:x['Tag'].values).reset_index(name='tags')
tags_df.head()

,Id,tags
0,1,"[bayesian, prior, elicitation]"
1,2,"[distributions, normality]"
2,3,"[software, open source]"
3,4,"[distributions, statistical significance]"
4,6,[machine learning]


In [36]:
# merge tags and questions
df = pd.merge(questions_df,tags_df,how='inner',on='Id')

In [37]:
df = df[['Id','Body','cleaned_text','tags']]
df.head()

,Id,Body,cleaned_text,tags
0,6,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",last year i read a blog post from brendan o connor entitled statistics vs machine learning fight that discussed some of the differences between the two fields andrew gelman responded favorably to ...,[machine learning]
1,21,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,what are some of the ways to forecast demographic census with some validation and calibration techniques some of the concerns census blocks vary in sizes as rural areas are a lot larger than conde...,"[forecasting, population, census]"
2,22,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,how would you describe in plain english the characteristics that distinguish bayesian from frequentist reasoning,"[bayesian, frequentist]"
3,31,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....",after taking a statistics course and then trying to help fellow students i noticed one subject that inspires much head desk banging is interpreting the results of statistical hypothesis tests it s...,"[hypothesis testing, t test, p value, interpretation, intuition]"
4,36,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",there is an old saying correlation does not mean causation when i teach i tend to use the following standard examples to illustrate this point number of storks and birth rate in denmark number of ...,"[correlation, teaching]"


In [ ]:
df.shape

(85085, 4)

There are over 85,000 unique questions and over 1300 tags.

# Dataset Preparation

In [38]:
# check frequency of occurence of each tag
freq= {}
for i in df['tags']:
  for j in i:
    if j in freq.keys():
      freq[j] = freq[j] + 1
    else:
      freq[j] = 1

In [20]:
freq

{'machine learning': 6089,
 'forecasting': 1422,
 'population': 219,
 'census': 32,
 'bayesian': 2656,
 'frequentist': 144,
 'hypothesis testing': 3869,
 't test': 1418,
 'p value': 1040,
 'interpretation': 860,
 'intuition': 168,
 'correlation': 2871,
 'teaching': 107,
 'nonparametric': 922,
 'survival': 1081,
 'hazard': 162,
 'time series': 5559,
 'garch': 302,
 'volatility forecasting': 64,
 'finance': 200,
 'data visualization': 1549,
 'references': 1076,
 'methodology': 142,
 'theory': 137,
 'classification': 2881,
 'information retrieval': 85,
 'text mining': 432,
 'communication': 40,
 'sample size': 847,
 'polling': 19,
 'r': 13236,
 'poisson': 865,
 'count data': 347,
 'epidemiology': 198,
 'regression': 10959,
 'outliers': 629,
 'scales': 248,
 'ordinal': 624,
 'likert': 366,
 'interval': 87,
 'software': 151,
 'standard deviation': 866,
 'variance': 1576,
 'anova': 2505,
 'random effects model': 569,
 'modeling': 1055,
 'spatial': 429,
 'clustering': 1952,
 'estimation': 153

Let's find out the most frequent tags.

In [39]:
# sort the dictionary in descending order
freq = dict(sorted(freq.items(), key=lambda x:x[1],reverse=True))

In [22]:
freq.items()

dict_items([('r', 13236), ('regression', 10959), ('machine learning', 6089), ('time series', 5559), ('probability', 4217), ('hypothesis testing', 3869), ('self study', 3732), ('distributions', 3501), ('logistic', 3316), ('classification', 2881), ('correlation', 2871), ('statistical significance', 2666), ('bayesian', 2656), ('anova', 2505), ('normal distribution', 2181), ('multiple regression', 2054), ('mixed model', 1998), ('clustering', 1952), ('neural networks', 1897), ('mathematical statistics', 1888), ('confidence interval', 1776), ('categorical data', 1703), ('generalized linear model', 1614), ('variance', 1576), ('data visualization', 1549), ('estimation', 1533), ('forecasting', 1422), ('t test', 1418), ('pca', 1395), ('sampling', 1363), ('cross validation', 1344), ('repeated measures', 1335), ('spss', 1296), ('svm', 1283), ('chi squared', 1261), ('maximum likelihood', 1209), ('predictive models', 1189), ('multivariate analysis', 1116), ('survival', 1081), ('references', 1076), (

In [44]:
# Top 10 most frequent tags
common_tags = list(freq.keys())[:10]
common_tags

['r',
 'regression',
 'machine learning',
 'time series',
 'probability',
 'hypothesis testing',
 'self study',
 'distributions',
 'logistic',
 'classification']

In [54]:
df.head(10)

,Id,Body,cleaned_text,tags
0,6,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",last year i read a blog post from brendan o connor entitled statistics vs machine learning fight that discussed some of the differences between the two fields andrew gelman responded favorably to ...,[machine learning]
1,21,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,what are some of the ways to forecast demographic census with some validation and calibration techniques some of the concerns census blocks vary in sizes as rural areas are a lot larger than conde...,"[forecasting, population, census]"
2,22,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,how would you describe in plain english the characteristics that distinguish bayesian from frequentist reasoning,"[bayesian, frequentist]"
3,31,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....",after taking a statistics course and then trying to help fellow students i noticed one subject that inspires much head desk banging is interpreting the results of statistical hypothesis tests it s...,"[hypothesis testing, t test, p value, interpretation, intuition]"
4,36,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",there is an old saying correlation does not mean causation when i teach i tend to use the following standard examples to illustrate this point number of storks and birth rate in denmark number of ...,"[correlation, teaching]"
5,93,"<p>We're trying to use a Gaussian process to model h(t) -- the hazard function -- for a very small initial population, and then fit that using the available data. While this gives us nice plots f...",we re trying to use a gaussian process to model h t the hazard function for a very small initial population and then fit that using the available data while this gives us nice plots for credible s...,"[nonparametric, survival, hazard]"
6,95,<p>I have been using various GARCH-based models to forecast volatility for various North American equities using historical daily data as inputs.</p>\n\n<p>Asymmetric GARCH models are often cited ...,i have been using various garch based models to forecast volatility for various north american equities using historical daily data as inputs asymmetric garch models are often cited as a modificat...,"[time series, garch, volatility forecasting, finance]"
7,103,<p>What is the best blog on data visualization?</p>\n\n<p>I'm making this question a community wiki since it is highly subjective. Please limit each answer to one link.</p>\n\n<hr>\n\n<p><strong>...,what is the best blog on data visualization i m making this question a community wiki since it is highly subjective please limit each answer to one link please note the following criteria for prop...,"[data visualization, references]"
8,113,"<p>I have been looking into theoretical frameworks for method selection (note: not model selection) and have found very little systematic, mathematically-motivated work. By 'method selection', I m...",i have been looking into theoretical frameworks for method selection note not model selection and have found very little systematic mathematically motivated work by method selection i mean a frame...,"[machine learning, methodology, theory]"
9,114,"<p>What statistical research blogs would you recommend, and why?</p>\n",what statistical research blogs would you recommend an

We will use only those questions/queries that have the above 10 tags associated with it.

In [51]:
x=[]
y=[]

for i in range(len(df['tags'])):

  temp=[]
  for j in df['tags'][i]:
    if j in common_tags:
      temp.append(j)


  if(len(temp)>1):
    x.append(df['cleaned_text'][i])
    y.append(temp)

In [43]:
temp

['hypothesis testing']

In [55]:
# number of questions left
len(x)

11106

In [48]:
y

[['r', 'time series'],
 ['regression', 'distributions'],
 ['distributions', 'probability', 'hypothesis testing'],
 ['hypothesis testing', 'self study'],
 ['r', 'regression', 'time series'],
 ['r', 'time series', 'self study'],
 ['probability', 'hypothesis testing'],
 ['r', 'regression'],
 ['r', 'regression'],
 ['regression', 'logistic'],
 ['r', 'regression', 'time series'],
 ['machine learning', 'classification'],
 ['r', 'hypothesis testing'],
 ['regression', 'time series'],
 ['time series', 'hypothesis testing'],
 ['machine learning', 'classification'],
 ['regression', 'logistic'],
 ['machine learning', 'logistic'],
 ['regression', 'probability', 'logistic'],
 ['hypothesis testing', 'self study'],
 ['r', 'regression', 'logistic'],
 ['r', 'regression', 'hypothesis testing'],
 ['r', 'machine learning'],
 ['machine learning', 'classification'],
 ['probability', 'self study'],
 ['probability', 'distributions'],
 ['machine learning', 'classification'],
 ['r', 'regression', 'logistic'],
 ['

In [ ]:
y[:10]

[['r', 'time series'],
 ['regression', 'distributions'],
 ['distributions', 'probability', 'hypothesis testing'],
 ['hypothesis testing', 'self study'],
 ['r', 'regression', 'time series'],
 ['r', 'time series', 'self study'],
 ['probability', 'hypothesis testing'],
 ['r', 'regression'],
 ['r', 'regression'],
 ['regression', 'logistic']]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

y = mlb.fit_transform(y)
y.shape

(11106, 10)

In [ ]:
y[0,:]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1])

In [ ]:
mlb.classes_

array(['classification', 'distributions', 'hypothesis testing',
       'logistic', 'machine learning', 'probability', 'r', 'regression',
       'self study', 'time series'], dtype=object)

We can now split the dataset into training set and validation set.

In [ ]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(x, y, test_size=0.2, random_state=0,shuffle=True)

# Text Representation

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#prepare a tokenizer
x_tokenizer = Tokenizer()

x_tokenizer.fit_on_texts(x_tr)

In [ ]:
x_tokenizer.word_index

{'the': 1,
 'i': 2,
 'to': 3,
 'a': 4,
 'of': 5,
 'is': 6,
 'and': 7,
 'in': 8,
 'l': 9,
 'x': 10,
 'for': 11,
 'that': 12,
 'data': 13,
 'this': 14,
 't': 15,
 'have': 16,
 'y': 17,
 'with': 18,
 'model': 19,
 'it': 20,
 'are': 21,
 'be': 22,
 'my': 23,
 'as': 24,
 'on': 25,
 'e': 26,
 'p': 27,
 'if': 28,
 'can': 29,
 'n': 30,
 'but': 31,
 'not': 32,
 'm': 33,
 'or': 34,
 'r': 35,
 'how': 36,
 'regression': 37,
 'c': 38,
 'am': 39,
 's': 40,
 'from': 41,
 'test': 42,
 'what': 43,
 'would': 44,
 'b': 45,
 'so': 46,
 'time': 47,
 'there': 48,
 'using': 49,
 'which': 50,
 'an': 51,
 'do': 52,
 'one': 53,
 'each': 54,
 'value': 55,
 'use': 56,
 'by': 57,
 'some': 58,
 'variables': 59,
 'like': 60,
 'variable': 61,
 'we': 62,
 'at': 63,
 'na': 64,
 'any': 65,
 'f': 66,
 'distribution': 67,
 'two': 68,
 'values': 69,
 'set': 70,
 'you': 71,
 'all': 72,
 'function': 73,
 'fit': 74,
 'd': 75,
 'beta': 76,
 'question': 77,
 'then': 78,
 'mean': 79,
 'me': 80,
 'know': 81,
 'where': 82,
 'when'

In [ ]:
len(x_tokenizer.word_index)

25312

There are around 25,000 tokens in the training dataset. Let's see how many tokens appear at least 5 times in the dataset.

In [ ]:
thresh = 3

cnt=0
for key,value in x_tokenizer.word_counts.items():
  if value>=thresh:
    cnt=cnt+1

print(cnt)

12574


Over 12,000 tokens have appeared three times or more in the training set.


In [ ]:
# prepare the tokenizer again
x_tokenizer = Tokenizer(num_words=cnt,oov_token='unk')

#prepare vocabulary
x_tokenizer.fit_on_texts(x_tr)

Now that we have encoded every token to an integer, let's convert the text sequences to integer sequences. After that we will pad the integer sequences to the maximum sequence length, i.e., 100.

In [ ]:
# maximum sequence length allowed
max_len = 100

#convert text sequences into integer sequences
x_tr_seq = x_tokenizer.texts_to_sequences(x_tr)
x_val_seq = x_tokenizer.texts_to_sequences(x_val)

#padding up with zero
x_tr_seq = pad_sequences(x_tr_seq,  padding='post', maxlen=max_len)
x_val_seq = pad_sequences(x_val_seq, padding='post', maxlen=max_len)

Since we are padding the sequences with zeros, we must increment the vocabulary size by one.

In [ ]:
#no. of unique words
x_voc_size = x_tokenizer.num_words + 1
x_voc_size

12575

In [ ]:
x_tr_seq[0]

array([1953, 5711,  416, 2023,    1,  226, 1747, 3740,  609,   43,  181,
       1953,  372,   19,  100,  416,    9, 1747, 3839,  238,   27,   27,
         27,   27,   27,   70,    6, 6919,    8, 1163,   70,    6,   43,
         43, 1802, 1802, 1802,   36,   36,   36,   36, 4308, 5410,    4,
        124,  592,  107,   22,    2, 1747, 4065,   27,   10, 1309,   10,
       6414,   10,  190,   10,  416,   10,   27,   10, 1309,   10, 6414,
         10,  190,   10,  416,   10,  456,  139,   15,    7,    2, 4610,
        164,   27,   10, 1309,   10, 6414,   10,  190,   10,  416,   10,
         27,   76,   27, 1309,   76,   27, 6414,   76,   27,  190,   76,
         27], dtype=int32)

# Model Building

In [ ]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *

### Define Model Architecture

In [ ]:
#sequential model
model = Sequential()

#embedding layer
model.add(Embedding(x_voc_size, 50, input_shape=(max_len,), mask_zero=True))

#rnn layer
model.add(SimpleRNN(128,activation='relu'))

#dense layer
model.add(Dense(128,activation='relu'))

#output layer
model.add(Dense(10,activation='sigmoid'))

Understand the output shape and no. of parameters of each layer:

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           628750    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               22912     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 669,464
Trainable params: 669,464
Non-trainable params: 0
_________________________________________________________________


Define the optimizer and loss:

In [ ]:
#define optimizer and loss
model.compile(optimizer='adam',loss='binary_crossentropy')

Define a callback - Model Checkpoint. Model Checkpoint is a callback used to save the best model during training.

In [ ]:
# checkpoint to save best model during training
mc = ModelCheckpoint("weights.best.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

### Train the Model

Lets train the model for 10 epochs with a batch size of 128:

In [ ]:
#train the model
model.fit(x_tr_seq, y_tr, batch_size=128, epochs=10, verbose=1, validation_data=(x_val_seq, y_val), callbacks=[mc])

Train on 8884 samples, validate on 2222 samples
Epoch 1/10
8884/8884 [==============================] - 7s 792us/step - loss: 0.5166 - val_loss: 0.4769

Epoch 00001: val_loss improved from inf to 0.47690, saving model to weights.best.hdf5
Epoch 2/10
8884/8884 [==============================] - 7s 744us/step - loss: 0.4648 - val_loss: 0.4564

Epoch 00002: val_loss improved from 0.47690 to 0.45641, saving model to weights.best.hdf5
Epoch 3/10
8884/8884 [==============================] - 7s 751us/step - loss: 0.4194 - val_loss: 0.4252

Epoch 00003: val_loss improved from 0.45641 to 0.42519, saving model to weights.best.hdf5
Epoch 4/10
8884/8884 [==============================] - 7s 746us/step - loss: 0.3931 - val_loss: 0.4107

Epoch 00004: val_loss improved from 0.42519 to 0.41074, saving model to weights.best.hdf5
Epoch 5/10
8884/8884 [==============================] - 7s 742us/step - loss: 0.3595 - val_loss: 0.3988

Epoch 00005: val_loss improved from 0.41074 to 0.39884, saving model to

# Model Predictions

Load the best model weights and now, the model is ready for the predictions

In [ ]:
# load weights into new model
model.load_weights("weights.best.hdf5")

#predict probabilities
pred_prob = model.predict(x_val_seq)

In [ ]:
pred_prob[0]

array([0.05661559, 0.01500756, 0.0809429 , 0.34682304, 0.14759117,
       0.00561154, 0.4962414 , 0.627706  , 0.0462845 , 0.24911207],
      dtype=float32)

The predictions are in terms of probabilities for each of the 10 tags. Hence we need to have a threshold value to convert these probabilities to 0 or 1.

Let's specify a set of candidate threshold values. We will select the threshold value that performs the best for the validation set.

In [ ]:
#define candidate threshold values
threshold  = np.arange(0,0.5,0.01)
threshold

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49])

Let's define a function that takes a threshold value and uses it to convert probabilities into 1 or 0.

In [ ]:
# convert probabilities into classes or tags based on a threshold value
def classify(pred_prob,thresh):
  y_pred_seq = []

  for i in pred_prob:
    temp=[]
    for j in i:
      if j>=thresh:
        temp.append(1)
      else:
        temp.append(0)
    y_pred_seq.append(temp)

  return y_pred_seq

In [ ]:
from sklearn import metrics
score=[]

#convert to 1 array
y_true = np.array(y_val).ravel()

for thresh in threshold:

    #classes for each threshold
    y_pred_seq = classify(pred_prob,thresh)

    #convert to 1d array
    y_pred = np.array(y_pred_seq).ravel()

    score.append(metrics.f1_score(y_true,y_pred))

In [ ]:
# find the optimal threshold
opt = threshold[score.index(max(score))]
opt

0.29

# Model Evaluation

In [ ]:
#predictions for optimal threshold
y_pred_seq = classify(pred_prob,opt)
y_pred = np.array(y_pred_seq).ravel()

In [ ]:
print(metrics.classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.81      0.85     17520
           1       0.49      0.67      0.56      4700

    accuracy                           0.78     22220
   macro avg       0.69      0.74      0.71     22220
weighted avg       0.81      0.78      0.79     22220



In [ ]:
y_pred = mlb.inverse_transform(np.array(y_pred_seq))
y_true = mlb.inverse_transform(np.array(y_val))

df = pd.DataFrame({'comment':x_val,'actual':y_true,'predictions':y_pred})

In [ ]:
df.sample(10)

,comment,actual,predictions
1523,i have a really basic understanding of how the package limma works in that it fits a linear model to each row sample in a micro array dataset what i do not understand is how to use a design matrix...,"(machine learning, r)","(logistic, r, regression)"
1846,i would like to have an lm prediction per country as a new column for df based on the models per each country that should be based on data frame df df read table text target birds wolfs country a ...,"(r, regression)","(r, regression, time series)"
763,i am trying to figure out the inner workings of the mob function in the party package i can t figure out how the splitting variable is selected when it is a categorical variable in the publication...,"(machine learning, regression)","(logistic, r, regression)"
771,what is the best method to determine the minimum number of training samples required for a classifier i am only comparing one classifier four class problem discriminant function analysis dfa with ...,"(classification, machine learning)","(classification, machine learning)"
2190,if this is a duplicate question please point to the right way but the similar questions i ve found here haven t been sufficiently similar suppose i estimate the model y alpha beta x u and find tha...,"(hypothesis testing, r, regression)","(r, regression, time series)"
490,i am trying to fit and forecast log returns of a price data using arima model in r for reproducibility data is provided here steps followed code and results obtained check for outliers package for...,"(r, time series)","(r, regression, time series)"
473,i am trying fit an arima model to stock returns i have reached a decent model using the aic criterion however the ljung box p value under a diagnostic plots are pretty weird the null hypothesis ge...,"(r, time series)","(r, regression, time series)"
689,if there are few predictors that are highly skewed among a larger set of predictors in case of a linear regression problem should a boxcox transformation be applied to only these few predictors or...,"(machine learning, regression)","(logistic, machine learning, r, regression)"
767,i am currently working at work on a project that attempts to predict an environmental change variable i am personally not a huge fan of the project but i still want to do the best job possible any...,"(machine learning, r)","(logistic, r, regression, time series)"
44,i measured one response variable y as a function of two measured independent variables x and x it is common practice in my field of research to transform y into another response variable y x y the...,"(r, regression)","(logistic, r, regression, time series)"


### Inference

In [ ]:
def predict_tag(comment):
  text=[]

  #preprocess
  text = [cleaner(comment)]

  #convert to integer sequences
  seq = x_tokenizer.texts_to_sequences(text)

  #pad the sequence
  pad_seq = pad_sequences(seq,  padding='post', maxlen=max_len)

  #make predictions
  pred_prob = model.predict(pad_seq)
  classes = classify(pred_prob,opt)[0]

  classes = np.array([classes])
  classes = mlb.inverse_transform(classes)
  return classes

In [ ]:
comment = "For example, in the case of logistic regression, the learning function is a Sigmoid function that tries to separate the 2 classes"

print("Comment:",comment)
print("Predicted Tags:",predict_tag(comment))

Comment: For example, in the case of logistic regression, the learning function is a Sigmoid function that tries to separate the 2 classes
Predicted Tags: [('classification', 'logistic', 'machine learning', 'regression')]
